In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
KERNEL_SIZE = (10, 10)

In [3]:
# input data init 
background = cv2.imread('input_images/fat_cat_resized.jpg')
gt_background = cv2.imread('input_images/input_image2.jpg')
gtb_blured = cv2.blur(gt_background, KERNEL_SIZE)
gtb_lab = cv2.cvtColor(gt_background, cv2.COLOR_BGR2Lab)
gtb_lab_blured = cv2.blur(gtb_lab, KERNEL_SIZE)
gtb_hsv = cv2.cvtColor(gt_background, cv2.COLOR_BGR2HSV)
gtb_hsv_blured = cv2.blur(gtb_hsv, KERNEL_SIZE)
gtb_gray = cv2.cvtColor(gt_background, cv2.COLOR_BGR2GRAY)
gtb_gray_blured = cv2.blur(gtb_gray, KERNEL_SIZE)
cap = cv2.VideoCapture('input_videos/test_video3.mp4')

In [4]:
# video writer configuration
w = cap.get(cv2.CAP_PROP_FRAME_WIDTH);
h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT); 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_videos/GT.mp4',fourcc, 30.0, (int(w),int(h)))

In [5]:
from colormath.color_diff import delta_e_cie2000
from colormath.color_objects import LabColor

# array of measures of changes
def ciede_distances(source_img1, source_img2):
    img1 = np.array(source_img1, dtype=np.int32)    
    img2 = np.array(source_img2, dtype=np.int32)  
    
    if img2.shape[0] != img1.shape[0] or img2.shape[1] != img1.shape[1]:
        raise Exception('Wrong image sizes')
        
    h = img1.shape[0]
    w = img1.shape[1]
    
    measures = np.empty(img1.shape)
    
    for y in range(0, h):
        for x in range(0, w):
            color1_lab = LabColor(img1[y,x][0], img1[y,x][1], img1[y,x][2])
            color2_lab = LabColor(img2[y,x][0], img2[y,x][1], img2[y,x][2])
            measures[y, x] = delta_e_cie2000(color1_lab, color2_lab)
              
    return measures

In [6]:
def euclidean_distances(source_img1, source_img2, weights=np.array([1,1,1])):
    img1 = np.array(source_img1, dtype=np.int32)    
    img2 = np.array(source_img2, dtype=np.int32)  
    img1 = img1 * weights
    img2 = img2 * weights
    
    if img2.shape[0] != img1.shape[0] or img2.shape[1] != img1.shape[1]:
        raise Exception('Wrong image sizes')
    
    measures = np.empty(img1.shape)
    measures = np.linalg.norm(img1 - img2, axis=-1)
    
    return measures

In [7]:
import math
def euclidean_exp(img_1, img_2):
    img1 = np.array(img_1, dtype=np.int32)    
    img2 = np.array(img_2, dtype=np.int32)  
    
    if img2.shape[0] != img1.shape[0] or img2.shape[1] != img1.shape[1]:
        raise Exception('Wrong image sizes')
    
    measures = np.empty(img1.shape[0:-1])
    
    h = img1.shape[0]
    w = img1.shape[1]
    
    for y in range(0, h):
            for x in range(0, w):
                d = (img1[y,x][0] - img2[y,x][0])**2 + (img1[y,x][1] - img2[y,x][1])**2 + (img1[y,x][2] - img2[y,x][2])**2
                measures[y, x] = math.sqrt(d)
              
    return measures


In [8]:
def distances_gray(img1, img2):
    if img2.shape[0] != img1.shape[0] or img2.shape[1] != img1.shape[1]:
        raise Exception('Wrong image sizes')

    measures = (img1 - img2)
    measures = (measures > 0) * measures
    normalized_measures = np.uint8(measures)
    
    return normalized_measures

In [9]:
def threshold(image, threshold):
    white = np.full(image.shape, 255.0)
    return (image > threshold) * white

In [10]:
def change_background(mask, image, frame):
    mask = cv2.blur(mask, (3,3))
    masked_frame = ((mask == 255.0) * frame)
    masked_image = ((mask != 255.0) * image)
    return masked_frame + masked_image

In [11]:
#weights=[0.5, 5, 5], thresh = 45 are good
def ground_thruth_lab_euclidean(cap):
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            frame_lab = cv2.cvtColor(frame, cv2.COLOR_BGR2Lab)
            frame_lab_blured = cv2.blur(frame_lab, KERNEL_SIZE)

            measures = euclidean_distances(gtb_lab_blured, frame_lab_blured, weights=np.array([0.3, 3.5, 3.5]))
            normalized_measures = threshold(measures, 30.0)
            normalized_measures = np.uint8(normalized_measures)
            fgmask = cv2.cvtColor(normalized_measures, cv2.COLOR_GRAY2BGR)

            result = change_background(fgmask, background, frame)
            
            out.write(result)

        else:
            break
            
    cap.release()
    out.release()

In [12]:
def ground_thruth_rgb_euclidean(cap):
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            frame_blured = cv2.blur(frame, KERNEL_SIZE)
            measures = euclidean_distances(gtb_blured, frame_blured)
            normalized_measures = threshold(measures, 95.0)
            normalized_measures = np.uint8(normalized_measures)
            fgmask = cv2.cvtColor(normalized_measures, cv2.COLOR_GRAY2BGR)

            out.write(fgmask)

        else:
            break
            
    cap.release()
    out.release()

In [13]:
def ground_thruth_hsv_euclidean(cap):
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            frame_hsv_blured = cv2.blur(frame_hsv, KERNEL_SIZE)
            measures = euclidean_distances(gtb_hsv_blured, frame_hsv_blured, weights=np.array([5, 1, 5]))
            normalized_measures = threshold(measures, 185.0)
            normalized_measures = np.uint8(normalized_measures)
            fgmask = cv2.cvtColor(normalized_measures, cv2.COLOR_GRAY2BGR)
            
            out.write(fgmask)

        else:
            break
            
    cap.release()
    out.release()

In [14]:
ground_thruth_lab_euclidean(cap)